In [1]:
#@title Install required libraries
# Install required libraries (uncomment if needed)
!pip install -q --upgrade git+https://github.com/unslothai/unsloth.git
!pip install -q PyPDF2
!pip install -q trl==0.8.0 datasets xformers
!pip install -q transformers


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from google.colab import files
uploaded = files.upload()

Saving 2501.12948v1.pdf to 2501.12948v1 (1).pdf
Saving dataset.md to dataset (1).md
Saving deepseekv3-cost-explained.md to deepseekv3-cost-explained (1).md
Saving deepseekv3-explained.md to deepseekv3-explained (1).md
Saving design-notes-3fs.md to design-notes-3fs (1).md
Saving open-source-week.md to open-source-week (1).md


In [3]:
!pip install trl==0.9.6

  Using cached trl-0.9.6-py3-none-any.whl.metadata (12 kB)
Using cached trl-0.9.6-py3-none-any.whl (245 kB)
  Attempting uninstall: trl
    Found existing installation: trl 0.8.0
    Uninstalling trl-0.8.0:
      Successfully uninstalled trl-0.8.0


In [4]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install --upgrade transformers


Looking in indexes: https://download.pytorch.org/whl/cpu
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.4 MB/s eta 0:00:00
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.5.1+cu124
    Uninstalling torchaudio-2.5.1+cu124:
      Successfully uninstalled torchaudio-2.5.1+cu124


In [5]:
!pip install --upgrade torchvision


In [6]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
#!pip install trl==0.9.6
#!pip install -q bitsandbytes
#!pip install unsloth_zoo

In [7]:
# %% [code] cell
import os
import PyPDF2
import nltk
from transformers import pipeline
from datasets import Dataset

In [8]:
# %% [code] cell
# Function to extract text from PDFs and Markdown files
def extract_text_from_files(paths):
    """Extract raw text from PDFs and .md files."""
    all_text = []
    for path in paths:
        if path.endswith(".pdf"):
            with open(path, 'rb') as f:
                reader = PyPDF2.PdfReader(f)
                text = ""
                for page in reader.pages:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + "\n"
                all_text.append(text)
        else:
            with open(path, 'r', encoding='utf-8') as f:
                text = f.read()
                all_text.append(text)
    return all_text

# List your file paths here
file_paths = [
    "/content/2501.12948v1.pdf",
    "/content/dataset.md",
    "/content/deepseekv3-cost-explained.md",
    "/content/deepseekv3-explained.md",
    "/content/design-notes-3fs.md",
    "/content/open-source-week.md",
]

# Extract texts from files
raw_texts = extract_text_from_files(file_paths)
print("Number of files read:", len(raw_texts))

Number of files read: 6


In [9]:
for path, text in zip(file_paths, raw_texts):
    print("File:", path)
    print("Content snippet:", text[:300])  #SHOW THE DATA WHAT INSIDWE
    print("-" * 80)


File: /content/2501.12948v1.pdf
Content snippet: DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via
Reinforcement Learning
DeepSeek-AI
research@deepseek.com
Abstract
We introduce our first-generation reasoning models, DeepSeek-R1-Zero and DeepSeek-R1.
DeepSeek-R1-Zero, a model trained via large-scale reinforcement learning (RL) without su
--------------------------------------------------------------------------------
File: /content/dataset.md
Content snippet: # DualPipe
DualPipe is an innovative bidirectional pipeline parallelism algorithm introduced in the DeepSeek-V3 Technical Report. It achieves full overlap of forward and backward computation-communication phases, also reducing pipeline bubbles. For detailed information on computation-communication o
--------------------------------------------------------------------------------
File: /content/deepseekv3-cost-explained.md
Content snippet: <source name="https://medium.com/@visithkumarapperuma/deepseek-v3-a-game-changer-in

In [10]:
# %% [code] cell
# Improved sentence-based chunking that preserves full sentences and overlaps
def sentence_based_chunking_full(text, chunk_size=300, overlap_sentences=1):
    """
    Splits text into chunks based on full sentences.

    Each chunk contains full sentences such that the total word count
    is at least chunk_size (or as close as possible without breaking sentences).
    The last `overlap_sentences` from each chunk are repeated at the beginning
    of the next chunk to preserve context.
    """
    sentences = nltk.sent_tokenize(text)
    chunks = []
    i = 0
    while i < len(sentences):
        current_chunk = []
        current_word_count = 0
        # Add sentences until reaching or exceeding chunk_size
        while i < len(sentences) and current_word_count + len(sentences[i].split()) <= chunk_size:
            current_chunk.append(sentences[i])
            current_word_count += len(sentences[i].split())
            i += 1
        # If no sentence could be added (e.g., a very long sentence), force add one
        if not current_chunk and i < len(sentences):
            current_chunk.append(sentences[i])
            i += 1
        chunks.append(" ".join(current_chunk))
        # Move back by overlap_sentences to preserve context for the next chunk
        if overlap_sentences > 0 and i < len(sentences):
            i = max(i - overlap_sentences, 0)
    return chunks

In [11]:
!pip install nltk==3.8.1


In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
import time
for path, text in zip(file_paths, raw_texts):
    print(f"\nProcessing file: {path}")
    start_chunk_time = time.time()

    chunks = sentence_based_chunking_full(text, chunk_size=300, overlap_sentences=1)
    end_chunk_time = time.time()

    print(f"Total chunks: {len(chunks)}")
    print(f"Chunking took: {end_chunk_time - start_chunk_time:.2f} seconds")

    # Show first 2 chunks
    for idx, chunk in enumerate(chunks[:2]):
        print(f"\nChunk {idx+1} (first 500 characters):\n{chunk[:500]}")



Processing file: /content/2501.12948v1.pdf
Total chunks: 32
Chunking took: 0.10 seconds

Chunk 1 (first 500 characters):
DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via
Reinforcement Learning
DeepSeek-AI
research@deepseek.com
Abstract
We introduce our first-generation reasoning models, DeepSeek-R1-Zero and DeepSeek-R1. DeepSeek-R1-Zero, a model trained via large-scale reinforcement learning (RL) without super-
vised fine-tuning (SFT) as a preliminary step, demonstrates remarkable reasoning capabilities. Through RL, DeepSeek-R1-Zero naturally emerges with numerous powerful and intriguing
reasoning behav

Chunk 2 (first 500 characters):
. 5
2.2 DeepSeek-R1-Zero: Reinforcement Learning on the Base Model . . . . . . . . . . 5
2.2.1 Reinforcement Learning Algorithm . . . . . . . . . . . . . . . . . . . . . . 5
2.2.2 Reward Modeling . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6
2.2.3 Training Template . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [14]:
# %% [code] cell
from transformers import pipeline
# Initialize the summarization pipeline (using facebook/bart-large-cnn).
# Truncation is enabled when calling the pipeline, and we use a smaller chunk size.
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


In [ ]:
######################################################testcode

import os
# Set environment variables for debugging and to specify the GPU.
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # Enables synchronous CUDA error reporting

import torch
print("CUDA device count:", torch.cuda.device_count())
torch.cuda.set_device(0)

from transformers import pipeline

# Try loading the summarization pipeline on GPU.
try:
    print("Loading summarization model on GPU...")
    summarizer = pipeline("summarization", model="google/pegasus-xsum", device=0)
    print("Model loaded successfully!")
except Exception as e:
    print(f"An error occurred while loading the model on GPU: {e}")
    # As a fallback, try loading on CPU.
    print("Loading model on CPU instead...")
    summarizer = pipeline("summarization", model="google/pegasus-xsum", device=-1)
    print("Model loaded on CPU.")

# Test the summarizer with a simple input.
test_text = "Deep learning models are transforming natural language processing."
try:
    summary_output = summarizer(test_text, max_length=50, min_length=10, do_sample=False)
    print("Summary output:", summary_output)
except Exception as e:
    print(f"Error during summarization: {e}")


CUDA device count: 1
Loading summarization model on GPU...


Device set to use cuda:0
Your max_length is set to 50, but your input_length is only 11. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=5)


Model loaded successfully!
Summary output: [{'summary_text': 'Deep learning models are transforming natural language processing.'}]


In [16]:
# %% [code] cell
import time
import nltk

# Ensure 'punkt' is downloaded for sentence tokenization.
nltk.download('punkt')

def sentence_based_chunking_full_debug(text, chunk_size=150, overlap_sentences=1):
    """
    Splits text into chunks based on full sentences, printing debug information
    after processing each sentence. The overlap is applied only if more than one
    sentence is added to the chunk, preventing infinite loops on a single long sentence.

    Args:
        text (str): The input text.
        chunk_size (int): Target number of words per chunk.
        overlap_sentences (int): Number of whole sentences to overlap between chunks.

    Returns:
        List[str]: A list of sentence-based text chunks.
    """
    sentences = nltk.sent_tokenize(text)
    total_sentences = len(sentences)
    print(f"[DEBUG] Total sentences in this text: {total_sentences}")

    chunks = []
    i = 0
    while i < total_sentences:
        current_chunk = []
        current_word_count = 0

        # Add sentences until reaching or exceeding chunk_size.
        while i < total_sentences and current_word_count + len(sentences[i].split()) <= chunk_size:
            print(f"[DEBUG] Adding sentence {i+1}/{total_sentences} (length: {len(sentences[i].split())} words)")
            print(f"        Sentence preview: {sentences[i][:60]}...")
            current_chunk.append(sentences[i])
            current_word_count += len(sentences[i].split())
            i += 1

        # If no sentence could be added (e.g., a very long sentence), force-add one.
        if not current_chunk and i < total_sentences:
            print(f"[DEBUG] Forcing add of a very long sentence {i+1}/{total_sentences}")
            current_chunk.append(sentences[i])
            current_word_count += len(sentences[i].split())
            i += 1

        joined_chunk = " ".join(current_chunk)
        if joined_chunk.strip():
            chunks.append(joined_chunk)
            print(f"[DEBUG] Finished chunk with {current_word_count} words. Total chunks so far: {len(chunks)}\n")

        # Apply overlap only if more than one sentence was added.
        if overlap_sentences > 0 and i < total_sentences and len(current_chunk) > 1:
            old_i = i
            i = max(i - overlap_sentences, 0)
            print(f"[DEBUG] Overlapping by {overlap_sentences} sentence(s). Index changed from {old_i} to {i}.\n")

    print(f"[DEBUG] Created {len(chunks)} chunks from {total_sentences} sentences.\n")
    return chunks

# %% [code] cell
# Real-time debug processing for all files
all_chunks = []
file_chunk_counts = []
start_time = time.time()

for file_idx, text in enumerate(raw_texts):
    print(f"\n[DEBUG] Processing file {file_idx+1}/{len(raw_texts)}...")
    file_start = time.time()

    file_chunks = sentence_based_chunking_full_debug(text, chunk_size=150, overlap_sentences=1)
    all_chunks.extend(file_chunks)
    file_chunk_counts.append(len(file_chunks))

    file_end = time.time()
    print(f"[DEBUG] File {file_idx+1} produced {len(file_chunks)} chunks in {file_end - file_start:.2f} seconds.")
    print(f"[DEBUG] Total chunks so far: {len(all_chunks)}\n")

end_time = time.time()
print("Total chunks created:", len(all_chunks))
print(f"[DEBUG] Total processing time: {end_time - start_time:.2f} seconds")



[DEBUG] Processing file 1/6...
[DEBUG] Total sentences in this text: 921
[DEBUG] Adding sentence 1/921 (length: 21 words)
        Sentence preview: DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via
...
[DEBUG] Adding sentence 2/921 (length: 22 words)
        Sentence preview: DeepSeek-R1-Zero, a model trained via large-scale reinforcem...
[DEBUG] Adding sentence 3/921 (length: 12 words)
        Sentence preview: Through RL, DeepSeek-R1-Zero naturally emerges with numerous...
[DEBUG] Adding sentence 4/921 (length: 11 words)
        Sentence preview: However, it encounters challenges such as poor readability, ...
[DEBUG] Adding sentence 5/921 (length: 21 words)
        Sentence preview: To address these issues and further enhance reasoning perfor...
[DEBUG] Adding sentence 6/921 (length: 10 words)
        Sentence preview: DeepSeek-
R1 achieves performance comparable to OpenAI-o1-12...
[DEBUG] Adding sentence 7/921 (length: 27 words)
        Sentence preview: To support the
re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
# %% [code] cell
print("Total number of chunks created:", len(all_chunks))
print("\n--- Viewing Chunks ---\n")
# For each chunk, print its index and the first 200 characters (or the whole chunk if short)
for idx, chunk in enumerate(all_chunks):
    print(f"--- Chunk {idx+1} (first 200 characters) ---")
    print(chunk[:200])
    print("\n")


Total number of chunks created: 163

--- Viewing Chunks ---

--- Chunk 1 (first 200 characters) ---
DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via
Reinforcement Learning
DeepSeek-AI
research@deepseek.com
Abstract
We introduce our first-generation reasoning models, DeepSeek-R1-Zero and D


--- Chunk 2 (first 200 characters) ---
To support the
research community, we open-source DeepSeek-R1-Zero, DeepSeek-R1, and six dense models
(1.5B, 7B, 8B, 14B, 32B, 70B) distilled from DeepSeek-R1 based on Qwen and Llama. AIME 2024
(Pass@


--- Chunk 3 (first 200 characters) ---
. 4
2 Approach 5
2.1 Overview . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5
2.2 DeepSeek-R1-Zero: Reinforcement Learning on the Base Model . . . . . . . . . . 5


--- Chunk 4 (first 200 characters) ---
. . . . . . . . . . . . . . . . . . 6
2.2.4 Performance, Self-evolution Process and Aha Moment of DeepSeek-R1-Zero 6
2.3 DeepSeek-R1: Reinforcement Learning with Cold Start .

In [18]:
from transformers import pipeline

# Create a summarization pipeline for CPU only.
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=-1)

synthetic_data = []

for i, chunk in enumerate(all_chunks):
    question = "What is the main idea of the following text?\n\n" + chunk
    summary_text = ""
    try:
        summary_output = summarizer(
            chunk,
            max_length=130,
            min_length=30,
            do_sample=False,
            truncation=True,
        )
        summary_text = summary_output[0]['summary_text']
    except Exception as e:
        print(f"Error processing chunk {i} on CPU: {e}")
        continue  # Skip this chunk if summarization fails

    answer = "The text is about: " + summary_text

    conversation = [
        {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
        {"role": "user", "content": question},
        {"role": "assistant", "content": answer},
    ]

    synthetic_data.append({"conversations": conversation})

    # Print the conversation output for inspection.
    print(f"Conversation for chunk {i}:")
    for turn in conversation:
        print(f"{turn['role'].capitalize()}: {turn['content'][:200]}")  # print first 200 characters for brevity
    print("\n" + "="*80 + "\n")

print("Synthetic QA pairs created:", len(synthetic_data))


Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Conversation for chunk 0:
System: You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
User: What is the main idea of the following text?

DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via
Reinforcement Learning
DeepSeek-AI
research@deepseek.com
Abstract
We introduce our first-gener
Assistant: The text is about: DeepSeek-R1 achieves performance comparable to OpenAI-o1-1217 on reasoning tasks. To support the research community, we open-source the models and six dense models distilled from th


Conversation for chunk 1:
System: You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
User: What is the main idea of the following text?

To support the
research community, we open-source DeepSeek-R1-Zero, DeepSeek-R1, and six dense models
(1.5B, 7B, 8B, 14B, 32B, 70B) distilled from DeepSee
Assistant: The text is about: We open-source DeepSeek-R1-Zero, DeepSeeks-V3, and six dense models. The models are distilled from DeepSeeker-R 1 based on Qwen and Llam

Your max_length is set to 130, but your input_length is only 92. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)


Conversation for chunk 37:
System: You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
User: What is the main idea of the following text?

For harmlessness, we evaluate the entire
response of the model, including both the reasoning process and the summary, to identify and
mitigate any potenti
Assistant: The text is about: The integration of reward signals and diverse data distributions enables us to train a model that excels in reasoning while prioritizing helpfulness and harmlessness. For harmlessne


Conversation for chunk 38:
System: You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
User: What is the main idea of the following text?

We select Llama-3.3 because its
reasoning capability is slightly better than that of Llama-3.1. For distilled models, we apply only SFT and do not include
Assistant: The text is about: For distilled models, we apply only SFT and do not include an RL stage. Our primary goal here is todemonstrate the effectiveness of th

Your max_length is set to 130, but your input_length is only 98. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=49)


...

Conversation for chunk 145:
System: You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
User: What is the main idea of the following text?

If a storage target has medium failure, the related service sets the target’s local state to offline in heartbeat. If a storage service is down, storage t
Assistant: The text is about: If a storage service is down, storage targets managed by the service are marked offline. If a storage target has medium failure, the related service sets the target’s local state to




Your max_length is set to 130, but your input_length is only 46. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)


Conversation for chunk 146:
System: You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
User: What is the main idea of the following text?

-   The chain version is incremented if the chain is updated. -   If a storage target is marked offline, it’s moved to the end of chain. -   If a storage 
Assistant: The text is about: If a storage service finds public state of any local storage target is lastsrv or offline, it exits immediately. The service may be isolated from the cluster manager by network part


Conversation for chunk 147:
System: You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
User: What is the main idea of the following text?

-   Once the data recovery of a storage target in syncing state is completed, the storage service set the target’s local state to up-to-date in subsequent
Assistant: The text is about: Once the data recovery of a storage target in syncing state is completed, the storage service set the target’s local state to up-to-

Your max_length is set to 130, but your input_length is only 57. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=28)


Conversation for chunk 160:
System: You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
User: What is the main idea of the following text?

🔗 DeepGEMM GitHub Repo
⚡ Up to 1350+ FP8 TFLOPS on Hopper GPUs
✅ No heavy dependency, as clean as a tutorial
✅ Fully Just-In-Time compiled
✅ Core logic at
Assistant: The text is about: DeepGEMM GitHub Repo can run up to 1350+ FP8 TFLOPS on Hopper GPUs. No heavy dependency, as clean as a tutorial. Core logic at ~300 lines - yet outperforms expert-tuned kernels acro


Conversation for chunk 161:
System: You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
User: What is the main idea of the following text?

🔗 GitHub Repo

## Day 5 - 3FS, Thruster for All DeepSeek Data Access

Fire-Flyer File System (3FS) - a parallel file system that utilizes the full bandwid
Assistant: The text is about: Fire-Flyer File System (3FS) is a parallel file system that utilizes the full bandwidth of modern SSDs and RDMA networks.


Conversa

In [19]:
from datasets import Dataset

dataset = Dataset.from_list(synthetic_data)
dataset


Dataset({
    features: ['conversations'],
    num_rows: 162
})

In [20]:
from unsloth import FastLanguageModel
import torch

try:
    model_name = "Qwen/Qwen2.5-3B-Instruct"
    load_in_4bit = True
    max_seq_length = 2048
    dtype = None

    print("Loading model...")
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_name,
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit,
    )
    print("Model loaded successfully!")
except Exception as e:
    print(f"An error occurred: {str(e)}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


<ipython-input-20-8fe0f863ecde>:1: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth Zoo will now patch everything to make training faster!
Loading model...
==((====))==  Unsloth 2025.3.9: Fast Qwen2 patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Model loaded successfully!


In [ ]:
from unsloth.chat_templates import get_chat_template

# Ensure the tokenizer is set up for Qwen2.5
tokenizer = get_chat_template(tokenizer, chat_template="qwen-2.5")

def format_qwen(examples):
    text = tokenizer.apply_chat_template(examples["conversations"], tokenize=False, add_generation_prompt=False)
    return {"text": text}

dataset = dataset.map(format_qwen, batched=False)


Map:   0%|          | 0/162 [00:00<?, ? examples/s]

In [22]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj","k_proj","v_proj","o_proj",
                      "gate_proj","up_proj","down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.3.9 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [23]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 50,
        max_steps = 200,  # Increase for real usage
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs_qwen3b",
        report_to = "none",
    ),
)

# Optionally only train on assistant outputs
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

trainer.train()

/content/unsloth_compiled_cache/UnslothSFTTrainer.py:569: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/content/unsloth_compiled_cache/UnslothSFTTrainer.py:583: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/content/unsloth_compiled_cache/UnslothSFTTrainer.py:607: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map (num_proc=2):   0%|          | 0/162 [00:00<?, ? examples/s]

/content/unsloth_compiled_cache/UnslothSFTTrainer.py:702: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `UnslothSFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(


Map (num_proc=2):   0%|          | 0/162 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 162 | Num Epochs = 5 | Total steps = 200
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 29,933,568/1,830,055,936 (1.64% trained)


Step,Training Loss
10,1.872100
20,1.039400
30,0.698400
40,0.363200
50,0.308300
60,0.402200
70,0.360600
80,0.244600
90,0.188800
100,0.231100


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=200, training_loss=0.3387684878706932, metrics={'train_runtime': 540.3464, 'train_samples_per_second': 1.481, 'train_steps_per_second': 0.37, 'total_flos': 4290125204226048.0, 'train_loss': 0.3387684878706932, 'epoch': 4.888888888888889})

In [38]:
from transformers import TextStreamer

test_prompt = [
    {"role": "user", "content": "tell me about MMLU,MMLU-Pro,andGPQADiamond,DeepSeek R1 achieves outstanding results"},
]

# Tokenize input and move to GPU
inputs = tokenizer.apply_chat_template(
    test_prompt, tokenize=True, add_generation_prompt=True, return_tensors="pt"
).to("cuda")

# Use a text streamer for real-time output
streamer = TextStreamer(tokenizer, skip_prompt=True)

print("Generating response...")
_ = model.generate(
    input_ids=inputs,
    max_new_tokens=128,
    temperature=1.2,
    top_p=0.9,
    streamer=streamer,
)


Generating response...
MMLU and MMLU-Pro are multiple-choice language understanding tasks. GPQADiamond is a question and answer task. DeepSeek R1 achieves outstanding results on these tasks.<|im_end|>


In [25]:
merged_16bit_dir = "qwen3b_finetuned_16bit"
model.save_pretrained_merged(
    merged_16bit_dir,
    tokenizer,
    save_method="merged_16bit",  # merges LoRA into base weights (float16)
)


Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.4G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 3.64 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 36/36 [00:00<00:00, 37.65it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving qwen3b_finetuned_16bit/pytorch_model-00001-of-00002.bin...
Unsloth: Saving qwen3b_finetuned_16bit/pytorch_model-00002-of-00002.bin...
Done.


In [26]:
# Ensure the 16-bit merged model path is correct
merged_16bit_dir = "qwen3b_finetuned_16bit"

# Export the model in 4-bit GGUF format
model.save_pretrained_gguf(
    merged_16bit_dir,      # ✅ Use the 16-bit merged model directory
    tokenizer=tokenizer,
    quantization_method="q4_k_m",  # ✅ 4-bit quantization
)

print("✅ 4-bit GGUF model saved in:", merged_16bit_dir)


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.06 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 36/36 [00:01<00:00, 26.16it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving qwen3b_finetuned_16bit/pytorch_model-00001-of-00002.bin...
Unsloth: Saving qwen3b_finetuned_16bit/pytorch_model-00002-of-00002.bin...
Done.


Unsloth: Converting qwen2 model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at qwen3b_finetuned_16bit into f16 GGUF format.
The output location will be /content/qwen3b_finetuned_16bit/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: qwen3b_finetuned_16bit
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00002.bin'
INFO:hf-to-gguf:token_embd.w

In [27]:
from google.colab import files
files.download("/content/qwen3b_finetuned_16bit/unsloth.Q4_K_M.gguf")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
!pip install llama-cpp-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 14.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.7-cp311-cp311-linux_x86_64.whl size=4601126 sha256=75b5268cff6362a29692475c62ff1a1d981eac50a853c6f25840c218e2d9dd15
  Stored in directory: /root/.cache/pip/wheels/eb/82/79/ac77fcd49324b75ae6aa18e63a87cf9da4371a57e2cdc8dc03
Successfully built llama-cpp-python


In [39]:
from llama_cpp import Llama

# Load the GGUF model
model_path = "/content/qwen3b_finetuned_16bit/unsloth.Q4_K_M.gguf"
llm = Llama(model_path=model_path, n_ctx=2048)  # Adjust n_ctx as needed

# Generate a response
prompt = "tell me about MMLU,MMLU-Pro,andGPQADiamond,DeepSeek R1 achieves outstanding results"
output = llm(prompt, max_tokens=100, stop=["\n"], echo=True)

# Print the output
print(output["choices"][0]["text"])


llama_model_loader: loaded meta data with 26 key-value pairs and 434 tensors from /content/qwen3b_finetuned_16bit/unsloth.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen2.5 3b Instruct Unsloth Bnb 4bit
llama_model_loader: - kv   3:                       general.organization str              = Unsloth
llama_model_loader: - kv   4:                           general.finetune str              = instruct-unsloth-bnb-4bit
llama_model_loader: - kv   5:                           general.basename str              = qwen2.5
llama_model_loader: - kv   6:                         general.size_label str              = 3B
lla

tell me about MMLU,MMLU-Pro,andGPQADiamond,DeepSeek R1 achieves outstanding results in MMLU and MMLU-Pro. The model is trained on 120K MMLU-Pro and 1M MMLU. GPQADiamond is a benchmark dataset for question answering on datasets like MMLU and MMLU-Pro. DeepSeek R1 achieves outstanding results on GPQADiamond. GPQADiamond is a benchmark dataset for question answering on datasets like MMLU and MMLU-Pro. DeepSeek R1 achieves outstanding results in M
